In [1]:
#!pip install pandas requests beautifulsoup4 seaborn

### Import modules

In [2]:
import re
import os
import csv
import json
import shutil
import pandas
import base64
import requests
import seaborn as sns
from pathlib import Path
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor

In [3]:
class StratNinja:
    def __init__(self, file_path, file_meta, save):
        self.file_path = file_path
        self.file_meta = file_meta
        self.save = save

        self.publics = []
        self.privates = []

        self.metadata = []
        self.processed = set()

        self.load_data()

    def load_data(self):
        self.metadata = []
        self.processed = set()

        if not self.file_meta or not os.path.exists(self.file_meta):
            return

        try:
            with open(self.file_meta, "r", encoding="utf-8") as f:
                for line in f:
                    line = line.strip()
                    if not line:
                        continue
                    try:
                        obj = json.loads(line)
                    except json.JSONDecodeError:
                        print(f"[WARN] Invalid line {self.file_meta}: {line[:80]}...")
                        continue
                    self.metadata.append(obj)
                    strat = obj.get("strategy")

                    if strat:
                        self.processed.add(strat)

        except Exception as e:
            print(f"load_data error: {e}")

    def save_data(self, row: dict):
        with open(self.file_meta, "a", encoding="utf-8") as f:
            f.write(json.dumps(row, ensure_ascii=False, separators=(",", ":")) + "\n")

        self.metadata.append(row)
        
        if row.get("strategy"):
            self.processed.add(row["strategy"])
    
    def get_public_strategies(self):
        resp = requests.request('GET', 'https://strat.ninja/strats.php')
        if resp.status_code == 200:
            rows = resp.text.splitlines()

            for row in rows:
                if re.search(r'target="_blank"', row):
                    match = re.search(r'href="overview.php\?strategy=(.*?)"', row)
                    if match:
                        name = match.group(1)
                        if 'private' in row.lower():
                            self.privates.append(name)
                        else:
                            self.publics.append(name)
            
    def get_user_strategies(self):
        pass

    def process_strategies(self, strategies):
        for index, strategy in enumerate(strategies):
            if strategy in self.processed:
                print(f"[{len(self.publics)}/{index + 1}] {strategy} skipping...")
                continue
            else:
                print(f"[{len(self.publics)}/{index + 1}] {strategy}")

            resp_info, tags = self.download_strategy_info(strategy)
            resp_code, code = self.download_strategy_code(strategy, self.save) if strategy not in self.privates else "", 404
            
            if not resp_code or not resp_info:
                continue
            
            scope = 'Public' if strategy in self.publics else 'Private'
            mode = self.get_mode(tags)
            timeframe = self.get_timeframe(tags)
            failed = self.get_failed(tags)
            bias = self.get_bias(tags)
            stalled = self.get_stalled(tags)
            leverage = self.get_leverage(tags)
            profit = self.get_profit(resp_info)
            short = self.get_short(resp_code)
            inds_set = self.get_indicators(resp_code)

            row = {
                "strategy": strategy,
                "scope": scope,
                "mode": mode,
                "timeframe": timeframe,
                "failed": failed,
                "bias": bias,
                "stalled": stalled,
                "leverage": leverage,
                "short": short,
                "profit": profit,
            }
            row.update({ind: 1 for ind in inds_set})
            self.save_data(row)

    def download_strategy_info(self, strategy):
        try:
            resp = requests.request("GET", f"https://strat.ninja/overview.php?strategy={strategy}")

            soup = BeautifulSoup(resp.text, features="html.parser")
            tags = soup.find("div", class_="tags")

            elements = []
            for tag in tags.find_all("a"):
                if not tag.find("img") and not tag.get("onclick"):
                    elements.append(tag.get_text())

            return resp.text, elements
        except Exception as e:
            print(f"EXCEPTION {strategy}: {e}")
            return None, None
    
    def download_strategy_code(self, strategy, save=False):
        try:
            resp = requests.request("GET", f'https://strat.ninja/mirror/{strategy}.py')

            if save:
                file_path = Path(self.file_path) / f"{strategy}.py"
                with open(file_path, "w", encoding="utf-8") as f:
                    f.write(resp.text)

            return resp.text, resp.status_code
        except Exception as e:
            print(f"EXCEPTION {strategy}: {e}")
            return None, None

    def get_mode(self, tags):
        modes = [ "Spot", "Futures"]
        for mode in modes:
            if mode in tags:
                return mode
        return None

    def get_timeframe(self, tags):
        timeframes = [ "1m", "3m", "5m", "10m", "15m", "30m", "1h", "2h", "4h", "6h", "12h", "1d", "1w", ]
        for timeframe in timeframes:
            if timeframe in tags:
                return timeframe
        return None

    def get_failed(self, tags):
        fails = [ "Failed" ]
        for fail in fails:
            if fail in tags:
                return fail
        return None

    def get_bias(self, tags):
        biases = ["Biased (Lookahead Analysis)", "Bias unchecked", "Unbiased"]
        for bias in biases:
            if bias in tags:
                return bias
        return None

    def get_stalled(self, tags):
        stalleds = [ "Stalled - 90 Percent Negative", "Stalled - Biased", "Stalled - Negative", ]
        for stalled in stalleds:
            if stalled in tags:
                return stalled
        return None
        
    def get_leverage(self, tags):
        leverage = "X"
        if tags[-1].endswith(leverage):
            return tags[-1]
        return None
    
    def get_short(self, resp):
        can_short = re.search(r'can_short\s*=\s*True', str(resp))
        if can_short:
            return True
        return False
    
    def get_indicators(self, resp):
        indicators = []
        matches = re.findall(r'dataframe\[(.*?)\]', str(resp))
        for match in matches:
            parts = match.split(',') if ',' in match else [match]
            for p in parts:
                clean = re.sub(r"[\'\"\\/\s]", "", p)
                if clean:
                    indicators.append(clean.lower())

        return indicators
    
    def get_profit(self, resp):
        profit = 0
        cum_prof = []
        
        if 'Failed' not in resp:
            soup = BeautifulSoup(resp, features="html.parser")

            table = soup.find('table', id='example')
            if not table: return

            tbody = table.find('tbody')
            if not tbody: return

            rows = tbody.find_all('tr')
            for row in rows:
                columns = row.find_all('th')
                cum_prof.append(float(columns[5].text))

            if cum_prof:
                profit = sum(cum_prof) / len(cum_prof)

        return profit
    

### Data Loading

In [4]:
if not os.path.exists('raw.csv'):
    sn = StratNinja(
        file_path = "strategies",
        file_meta = "strategies_metadata.ndjson",
        save=True
    )

    # Process specified strategies
    # sn.process_strategies(['01_CombinedBinHAndClucV7_OPT', 'ZaratustraV13', 'NDrop_3',])

    # Process all public strategies
    sn.get_public_strategies()
    sn.process_strategies(sn.publics)

    df = pandas.DataFrame(sn.metadata)
    df.columns = df.columns.str.replace('[', '', regex=False)
    df.to_csv('raw.csv')
    df
else:
    df = pandas.read_csv('raw.csv')
    df

/tmp/ipykernel_173995/2767535811.py:20: DtypeWarning: Columns (7,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pandas.read_csv('raw.csv')


### Spot

In [5]:
spots = df[df["mode"] == "Spot"]
spots = spots[spots["bias"] == "Unbiased"]
spots = spots[spots["stalled"].isna()]

spots.sort_values("profit", ascending=False).head(25)

,Unnamed: 0,strategy,scope,mode,timeframe,failed,bias,stalled,leverage,short,...,bbm_5m,close_btc_5m,aup,ado,aup.1,ddi,lrs,rsi_15m,trend_ichimoku_base,trend_kst_diff
656,656,ClucHAnix_BB_RPB_MOD2,Public,Spot,1m,NaN,Unbiased,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
21,21,abbas,Public,Spot,5m,NaN,Unbiased,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
666,666,ClucHAnix_BB_RPB_MOD_CTT,Public,Spot,1m,NaN,Unbiased,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1701,1701,EI3v2_tag_cofi_green_3474790687_mod7_zema,Public,Spot,5m,NaN,Unbiased,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
662,662,ClucHAnix_BB_RPB_MOD_2,Public,Spot,1m,NaN,Unbiased,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1688,1688,EI3v2_tag_cofi_green,Public,Spot,5m,NaN,Unbiased,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1773,1773,ElliotV8_original_ichiv2_2,Public,Spot,5m,NaN,Unbiased,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1783,1783,ElliotV8_original_ichiv3_855,Public,Spot,5m,NaN,Unbiased,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2835,2835,NotAnotherSMAOffsetStrategyHOv3_akiva,Public,Spot,5m,NaN,Unbiased,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
642,642,ClucHAnix_6,Public,Spot,1m,NaN,Unbiased,NaN,NaN,False,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Futures

In [6]:
futures = df[df["mode"] == "Futures"]
futures = futures[futures["bias"] == "Unbiased"]
futures = futures[futures["stalled"].isna()]

futures.sort_values("profit", ascending=False).head(25)

,Unnamed: 0,strategy,scope,mode,timeframe,failed,bias,stalled,leverage,short,...,bbm_5m,close_btc_5m,aup,ado,aup.1,ddi,lrs,rsi_15m,trend_ichimoku_base,trend_kst_diff
3127,3127,RsiquiV3,Public,Futures,5m,NaN,Unbiased,NaN,38X,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3126,3126,RsiquiV2,Public,Futures,5m,NaN,Unbiased,NaN,10X,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3916,3916,ZTV16,Public,Futures,5m,NaN,Unbiased,NaN,10X,True,...,NaN,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
3128,3128,RsiquiV3_2,Public,Futures,5m,NaN,Unbiased,NaN,10X,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3904,3904,ZaratustraV8,Public,Futures,5m,NaN,Unbiased,NaN,10X,True,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3129,3129,RsiquiV4,Public,Futures,5m,NaN,Unbiased,NaN,5X,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1717,1717,el,Public,Futures,5m,NaN,Unbiased,NaN,10X,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3882,3882,ZaratustraV13,Public,Futures,5m,NaN,Unbiased,NaN,10X,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3907,3907,ZarTest02,Public,Futures,5m,NaN,Unbiased,NaN,10X,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3269,3269,SlopeV5,Public,Futures,15m,NaN,Unbiased,NaN,10X,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
